In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from bs4 import BeautifulSoup
import requests
import re


In [1]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [ ]:
# state compactness averages scrape
site = 'https://www.governing.com/gov-data/politics/gerrymandered-congressional-districts-compactness-by-state.html'

response = requests.get(site)

page = response.text
soup = BeautifulSoup(page, 'html5')
tab = soup.find_all('tr')

In [ ]:
#finding beginning and end of table
tab[2]

In [ ]:
tab[44]

In [ ]:
# create dictionary to set up dataframe
comp = {}

for row in tab[2:45]:
    items = row.find_all('td')
    state = items[0].text
    comp[state] = [i.text for i in items[1:]]

In [ ]:
comp_scores = pd.DataFrame(comp).T
comp_scores.columns = ['districts', 'polsby-popper', 'schwartzberg', 'convex hull', 'reock']

comp_scores.head()
comp_scores.columns

In [ ]:
comp_scores['state'] = comp_scores.index

In [ ]:
compdf = comp_scores.drop(['districts'], axis = 1)

In [ ]:
compdf.to_pickle('comp_table')

In [ ]:
# wiki table for district areas
site = 'https://en.wikipedia.org/wiki/List_of_United_States_congressional_districts'
response = requests.get(site)

page = response.text
soup = BeautifulSoup(page, 'html5')

In [ ]:
tab = soup.find_all('tbody')
areas_raw = tab[1]
areas = areas_raw.find_all('tr')
areas[0]

In [ ]:
#dictionary for rows
rows = {}

for row in areas[1:]:

    items = row.find_all('td')
    state = items[1].text
    rank = items[0]
    rows[state] = [rank] + [i.text for i in items]

In [ ]:
#init dataframe
district_df = pd.DataFrame(rows).T

In [ ]:
district_df.columns = ['rank1', 'rank', 'district','areami', 'areakm']
district_df = district_df.rename(columns={'district' :'desc'})


In [ ]:
# remove newlines \n from columns
district_df['rank'] = [item.strip() for item in district_df['rank'] if str(item)]
district_df['desc'] = [item.strip() for item in district_df['desc'] if str(item)]
district_df['areami'] = [item.strip() for item in district_df['areami'] if str(item)]
district_df['areakm'] = [item.strip() for item in district_df['areakm'] if str(item)]

In [ ]:
district_df = district_df.drop(['rank1'], axis=1)

In [ ]:
tb = district_df

In [ ]:
# prepare dis columns for xx-oo format
tb['state'] = tb['desc'].apply(lambda x: ' '.join((x.split()[:-1])))
tb['disno'] = tb['desc'].apply(lambda x: x.split()[-1])
tb['disno'] = tb['disno'].apply(lambda x: x[:-2])
tb['disno'] = tb['disno'].apply(lambda x: x.replace('at-lar', 'AL'))

In [ ]:
#remove nonstates
tb['state'].unique()
filterlist = ['District of Columbia', 'American Samoa', 'US Virgin Islands', 'N. Mariana Islands', 'Guam']

tbdrop = tb[~tb['state'].isin(filterlist)]

In [ ]:
tbdrop['st'] = tbdrop['state'].apply(lambda x: us_state_abbrev[x])
# district xx-00
tbdrop['district'] = tbdrop['st'] + '-' + tbdrop['disno']

In [ ]:

tbend = tbdrop.iloc[:, [2, 4, 7]]

In [ ]:
# change 0 to AL
tbend = tbend.replace({'district': {'WY-0': 'WY-AL', 'AK-0': 'AK-AL', 'MT-0': 'MT-AL', 'SD-0':'SD-AL', 'ND-0':'ND-AL', 'DE-0':'DE-AL', 'VT-0':'VT-AL'}})

In [ ]:
tbend.to_pickle('area_table')

In [ ]:
# CSV from cookpolitical.com for Partisan Voting Index, margins
df = pd.read_csv('pvitracker.csv')

In [ ]:
dfdrop = df.drop(axis=0, index=[0, 1])
#drop index rows

# change state abbreviations to XX-## format
dfdrop['abbv'] = dfdrop['State'].apply(lambda x: us_state_abbrev[x])
dfdrop['district'] = dfdrop['abbv'] + '-' + dfdrop['CD#']
dfdropp = dfdrop.copy()

In [ ]:
# convert republican margins to negative number, opposite of positive dem margins
dfdropp['2018 Cook PVI Score'] = dfdropp['2018 Cook PVI Score'].apply(lambda x: ('-' + x[2:]) if x[0] == 'R' else 0 if x == 'EVEN' else x[2:])
dfdropp.rename(columns={'2018 Cook PVI Score': 'pvi'}, inplace=True)

#columns to keep
dfend = dfdropp[['pvi', 'Dem Margin', '2016 Clinton Margin', 'district']]

In [ ]:
# formatting column to convert to numeric form
dfend['Dem Margin'] = dfend['Dem Margin'].apply(lambda x: x[:-1])
dfend['2016 Clinton Margin'] = dfend['2016 Clinton Margin'].apply(lambda x: x[:-1])
dfend[['pvi','Dem Margin','2016 Clinton Margin']] = dfend[['pvi','Dem Margin','2016 Clinton Margin']].apply(pd.to_numeric)

In [ ]:
dfend.to_pickle('margins2018')

In [ ]:
# proximity.com csv for gini index, pop
df = pd.read_csv('https://api.census.gov/data/2018/acs/acs1?get=NAME,B01001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B03003_003E,B29001_001E,B19013_001E,B19301_001E,B25077_001E,B25064_001E,B19083_001E,B25001_001E,B25002_002E,B25003_002E,B25003_003E,B25002_003E&for=congressional%20district:*')

In [ ]:
#renaming columns in roundabout fashion
dfkeep[['state', 'population', 'gini', 'disno']] = dfkeep[['[["NAME"', 'B01001_001E', 'B19083_001E', 'congressional district]']]
dfkeep.drop(['[["NAME"', 'B01001_001E', 'B19083_001E', 'congressional district]'], axis=1, inplace=True)

In [ ]:
#cleaning column data
dfkeep['disno'] = dfkeep['disno'].apply(lambda x: x.replace(']', ''))
dfkeep['state'] = dfkeep['state'].apply(lambda x: x.replace('"','').strip())
dfkeep['disno'] = dfkeep['disno'].apply(pd.to_numeric)

dfkeep['district'] = dfkeep['state'].map(us_state_abbrev) + '-' + dfkeep['disno'].map(str)
dfkeep.drop(['state', 'disno'],axis=1, inplace=True)

# converting at district '0' to At large districts 'AL'
dfkeep = dfkeep.replace({'district': {'WY-0': 'WY-AL', 'AK-0': 'AK-AL', 'MT-0': 'MT-AL', 'SD-0':'SD-AL', 'ND-0':'ND-AL', 'DE-0':'DE-AL', 'VT-0':'VT-AL'}})

In [ ]:
dfkeep.sort_values(by=['district'])

dfkeep.to_pickle('gini_table')

In [ ]:
# rural district stats 
df = pd.read_csv('urcd2010.csv', names=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])

In [ ]:
# keeping urban and rural percent columns
dfkeep = df.iloc[:,[0, 3, 8,12]]
dfkeep.rename(columns={1: 'state', 4: 'disno', 9: 'per_urban', 13: 'per_rural'}, inplace=True)

In [ ]:
# formatting district codes
dfkeep['district'] = dfkeep['state'] + '-' + dfkeep['disno'].map(str)
dfkeep.drop(['disno'],axis=1,inplace=True)

dfkeep = dfkeep.replace({'district': {'WY-0': 'WY-AL', 'AK-0': 'AK-AL', 'MT-0': 'MT-AL', 'SD-0':'SD-AL', 'ND-0':'ND-AL', 'DE-0':'DE-AL', 'VT-0':'VT-AL'}})

In [ ]:
dfkeep.to_pickle('rural_table')

In [ ]:
# Daily Kos candidate and demographic info
df=pd.read_csv('116cong.csv')


In [ ]:
dfkeep = df.iloc[:, [1,2,7,8,9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 28, 29, 30, 31, 32, 33, 42, 46, 50]]
dfkeep.columns

In [ ]:
dfkeep.rename(columns = {'2016 President': 'clinton2016', 'Unnamed: 15': 'trump2016', '2012 President': 'obama2012', 'Unnamed: 17': 'romney2012', '2018 US House': '2018dem', 'Unnamed: 21': '2018rep', '2016 US House': '2016dem', 'Unnamed: 23': '2016rep', '2014 US House': '2014dem', 'Unnamed: 25': '2014rep', '2014-2018 ACS Citizen Adult Population': 'whiteper', 'Unnamed: 29': 'blackper', 'Unnamed: 30': 'latinoper', 'Unnamed: 31': 'asianper', 'Unnamed: 32': 'nativeper', 'Unnamed: 33': 'other', 'Unnamed: 42': 'bach_higher', 'Unnamed: 46': 'median_income', 'Non-College White Share': 'noncol_white'}, inplace=True)

In [ ]:
dfdrop = dfkeep.drop(axis=0, index=[0, 1])
dfa = dfdrop.copy()
dfad = dfa

In [ ]:

# convery columns to numeric, missing values = null
dfad.iloc[:, [ 3, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]] = dfad.iloc[:, [ 3, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Past Election margins
dfad['2016presmarg'] = dfad['clinton2016'] - dfad['trump2016']
dfad['2012presmarg'] = dfad['obama2012'] - dfad['romney2012']
dfad['2016marg'] = dfad['2016dem'] - dfad['2016rep']
dfad['2014marg'] = dfad['2014dem'] - dfad['2014rep']


In [ ]:
# changing district code format
dfad['Code'] = dfad['Code'].apply(lambda x: x if x[3] != '0' else x[:3] + x[4])

#calculate ages
dfad['age'] = 2018 - dfad['Birth Year']

In [ ]:
a = dfad.sort_values(by=['First Elected'])

In [ ]:
# time in office / incumbent 
dfad['First Elected'] = dfad['First Elected'].apply(lambda x: str(x)[:4])
dfo = a.drop(axis=0, index=[366, 296, 120, 72, 129])
dfo['First Elected'] = dfo['First Elected'].apply(pd.to_numeric)
dfo['time_in_office'] = 2018 - dfo['First Elected']

dfo.sort_values(by=['time_in_office'],ascending = False)

In [ ]:
dfo.to_pickle('demo2018')

In [ ]:
# checking dummy categories
dfo['Pre-2018 Incumbent'].unique()

In [ ]:
dfo['Religion'].unique()

In [ ]:
# gerrymandering index for districts
cdf = pd.read_csv('gerryindex.csv')

In [ ]:
# fix district codes 
cdf['st'] = cdf['State'].apply(lambda x: us_state_abbrev[x])
cdf['district'] = cdf['st'] + '-' + cdf['District'].map(str)

In [ ]:
cdf = cdf.drop(['Unnamed: 0', 'State', 'District', 'st'],axis=1)

# replace 0 with AL for at large districts
cdf = cdf.replace({'district': {'WY-0': 'WY-AL', 'AK-0': 'AK-AL', 'MT-0': 'MT-AL', 'SD-0':'SD-AL', 'ND-0':'ND-AL', 'DE-0':'DE-AL', 'VT-0':'VT-AL'}})

In [ ]:
cdf.to_pickle('gerryindex')